In [1]:
from opentrons import protocol_api
import opentrons.execute

metadata = {
    'protocolName': 'Protocol Title',
    'author': 'Seong-Kun Bak <tjdrns27@kribb.re.kr>',
    'apiLevel': '2.11'   # CHECK IF YOUR API LEVEL HERE IS UP TO DATE
}

In [2]:
#Reset all of values ex) labware, instrument ..
protocol = opentrons.execute.get_protocol_api("2.11")

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [3]:
#after protocol.home is running, can't reset the tiprack's position
protocol.home()

Out of bounds move: X=(418.00000000000006 motor controller, 416.970344891251 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 416.970344891251 deck) too high for limit 418.0


### Lab ware

#### custom plate

In [4]:
import json
with open("nesteppendrf_96_wellplate_150ul.json") as labware_file:
    eppendorf = json.load(labware_file)
with open("bioneer_96_wellplate_300ul.json") as labware_file:
    bioneer = json.load(labware_file)

In [5]:
tag_primer_plate = protocol.load_labware_from_definition(bioneer, 2)
destination_plate = protocol.load_labware_from_definition(eppendorf, 1)
tiprack_20_sin = protocol.load_labware("opentrons_96_tiprack_20ul", 5)
tiprack_20_mul = protocol.load_labware("opentrons_96_tiprack_20ul", 6)

p20_sin = protocol.load_instrument("p20_single_gen2", "right", tip_racks=[tiprack_20_sin])
p20_mul = protocol.load_instrument("p20_multi_gen2", "left", tip_racks=[tiprack_20_mul])

#### tip position

In [6]:
#Input Tip start positions
p20_sin.starting_tip = tiprack_20_sin.well("G4")
p20_mul.starting_tip = tiprack_20_mul.well("A6")

### Liquid setup

In [11]:
# Set the Liquid list
reverse_tag = tag_primer_plate.rows()[0][0:8]
forward_tag_mix = destination_plate.columns()[-1]

#### dispense_reverse_tag

In [144]:
def reverse_tag_dispense(i):
    p20_sin.pick_up_tip()
    p20_sin.well_bottom_clearance.aspirate= -1
    p20_sin.mix(2,10, reverse_tag[i])
    p20_sin.aspirate(10, reverse_tag[i], rate=0.5)
    for j in destination_plate.columns()[i]:
        p20_sin.dispense(1, j)
    p20_sin.drop_tip()

In [145]:
for i in range(0, len(reverse_tag)):
    reverse_tag_dispense(i)

#### transfer_forward_mix

In [40]:
source = forward_tag_mix[0]
dest = destination_plate.columns()[7][0]

In [41]:
p20_mul.pick_up_tip()
p20_mul.well_bottom_clearance.aspirate = -2
p20_mul.aspirate(9, source, rate=0.5)

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [42]:
p20_mul.dispense(10, dest, rate=0.5)
p20_mul.mix(2, 5, dest, rate=0.5)
p20_mul.blow_out()
p20_mul.touch_tip()
protocol.delay(seconds = 1)
p20_mul.drop_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [20]:
p20_mul.pick_up_tip()


<InstrumentContext: p20_multi_v2.1 in LEFT>

In [25]:
p20_mul.return_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [38]:
p20_mul.drop_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [ ]:
# protocol.pause("Message") = Break until resume

In [175]:
p20_mul.return_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [172]:
p20_mul.pick_up_tip()

<InstrumentContext: p20_multi_v2.1 in LEFT>

In [57]:
# for starting position reset
p20_sin.reset_tipracks()

In [141]:
p20_sin.move_to(destination_plate["E8"].top())

<InstrumentContext: p20_single_v2.2 in RIGHT>

In [142]:
p20_sin.return_tip()

<InstrumentContext: p20_single_v2.2 in RIGHT>

In [139]:
p20_sin.drop_tip()

<InstrumentContext: p20_single_v2.2 in RIGHT>